In [1]:

import pandas as pd
import numpy as np
import copy
import pickle
import time
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
#Andy - not using Jupyter
util.startLoop()

#Andy- this is the default account, setting so that orders can be allocated to it.
# acc_num = 'DU1870227'
# acc_num = ''

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=10,timeout=100)


['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

<IB connected to 127.0.0.1:7497 clientId=10>

In [11]:
pip install BeautifulSoup4

     |████████████████████████████████| 115 kB 5.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
from bs4 import BeautifulSoup

In [2]:
security = Stock('AAPL', 'SMART', 'USD')

In [3]:
fundamentals = ib.reqFundamentalData(security, reportType='ReportSnapshot')
soup = BeautifulSoup(fundamentals)
market_cap = soup.find_all('ratio', fieldname="MKTCAP")

In [26]:
soup.find_all("issueid")

[<issueid type="Name">Ordinary Shares</issueid>,
 <issueid type="Ticker">AAPL</issueid>,
 <issueid type="RIC">AAPL.O</issueid>,
 <issueid type="DisplayRIC">AAPL.OQ</issueid>,
 <issueid type="InstrumentPI">331724</issueid>,
 <issueid type="QuotePI">7645713</issueid>,
 <issueid type="Name">Preference Shares</issueid>,
 <issueid type="InstrumentPI">100112850</issueid>]

In [48]:
ebitda = soup.find_all('ratio', fieldname="TTMEBITD")
gross_margin = soup.find_all('ratio', fieldname="TTMEBITD")
market_cap = soup.find_all('ratio', fieldname="MKTCAP")
beta = soup.find('ratio', fieldname='ProjPE')

In [73]:
#Read SP500
SP500_universe = pd.read_pickle("sp500univ.pickle")

In [74]:
SPExchangeData = pd.read_csv("SP500data.csv")
SPExchangeData.index= SPExchangeData["Stock"]

In [102]:
SPExchangeData[SPExchangeData.index=="GOOG"]

,Stock,primaryExchange
Stock,,
GOOG,GOOG,NASDAQ


In [ ]:
### SPExchangeData = pd.read_csv("SP500data.csv")
SPExchangeData.index= SPExchangeData["Stock"]
MCAP = pd.DataFrame()
for stock in SP500_universe:
    try:
        security = Stock(stock.replace("."," "), 'SMART', 'USD',primaryExchange=SPExchangeData.loc[stock]["primaryExchange"])
        fundamentals = ib.reqFundamentalData(security, reportType='ReportSnapshot')
        soup = BeautifulSoup(fundamentals)
        market_cap = soup.find_all('ratio', fieldname="MKTCAP")[0].string
        lista = {"stock":stock, "market_cap":market_cap}
        MCAP = MCAP.append(lista,ignore_index=True)
    except:
        print(stock)
    time.sleep(0.1)


In [97]:
MCAP.to_csv("marketcap.csv")

In [29]:
CoID = soup.find('CoID', Type='CompanyName')
gross_margin = soup.find('Ratio', FieldName='TTMGROSMGN')
ebitda = soup.find('Ratio', FieldName='TTMEBITD')



#pe_normalized = soup.find('Ratio', FieldName='APENORM').string
#beta = soup.find('Ratio', FieldName='BETA').string

In [31]:
ebitda

In [105]:
ib.reqMarketDataType(3)
element = "AAPL"
contract = Stock(element, 'SMART', 'USD')
print(ib.reqMktData(contract))
ib.reqMktData(contract)
request = str(ib.reqTickers(contract))
request = str(ib.reqMktData(contract))

Ticker(contract=Stock(symbol='AAPL', exchange='SMART', currency='USD'))


In [110]:
ib.reqMktData(contract).bid/ib.reqMktData(contract).ask

0.9999152087278482

Peer closed connection
